In [4]:
from flask import Flask, request, jsonify
from twilio. twiml.messaging_response import MessagingResponse
import os
from twilio.rest import Client


In [7]:
#account_sid = 'AC79c2fe6511ac0c9a1c3881c384798e22'
#auth_token = 'c9a5e13731974f3a5d548102fbd96629'
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='this is pretty cool',
  to='whatsapp:+85261520721'
)

print(message.sid)

SMf6243f8e62eab2a9a09d714284ec3032


In [ ]:


app = Flask(__name__)


@app.route("/whatsapp", methods=['POST'])
def receive_whatsapp_message():
    # Extracting the message SID, sender's number, and message body from the request
    message_sid = request.form.get('MessageSid', '')
    sender = request.form.get('From', '')
    message_body = request.form.get('Body', '')

    print(f"Message SID: {message_sid}, From: {sender}, Message: {message_body}")

    # Converting the message to a string (note: it's already a string)
    message_str = str(message_body)

    # You can process the message string here
    print(f"Received Message: {message_str}")

    # Responding to the sender via WhatsApp
    resp = MessagingResponse()
    resp.message(f"Received your message: {message_str}")

    return str(resp)


In [ ]:
if __name__ == "__main__":
    app.run(debug=True)